# read file

In [85]:
import pandas as pd
import numpy as np
import gzip
from gensim.parsing.preprocessing import remove_stopwords
import gensim
import os
data_dir = "data/Garden"

In [2]:
def parse(path):
    g = gzip.open(path, "rb")
    for l in g:
        yield eval(l)
        
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [25]:
data = getDF("data/Garden/reviews.json.gz")
data.head()

,unixReviewTime,reviewerName,summary,reviewerID,reviewText,helpful,reviewTime,asin,overall
0,1308614400,"Carter H ""1amazonreviewer@gmail . com""",Great Hoses,A1JZFGZEZVWQPY,Good USA company that stands behind their prod...,"[4, 4]","06 21, 2011",B00002N674,4.0
1,1402272000,"Darryl Bennett ""Fuzzy342""",Gilmour 10-58050 8-ply Flexogen Hose 5/8-Inch ...,A32JCI4AK2JTTG,This is a high quality 8 ply hose. I have had ...,"[0, 0]","06 9, 2014",B00002N674,5.0
2,1336176000,H B,Very satisfied!,A3N0P5AAMP6XD2,It's probably one of the best hoses I've ever ...,"[2, 3]","05 5, 2012",B00002N674,4.0
3,1373846400,Jason,Very high quality,A2QK7UNJ857YG,I probably should have bought something a bit ...,"[0, 0]","07 15, 2013",B00002N674,5.0
4,1375660800,jimmy,Good Hoses,AS0CYBAN6EM06,I bought three of these 5/8-inch Flexogen hose...,"[1, 1]","08 5, 2013",B00002N674,5.0


# Matching user, item

In [27]:
item_unique = sorted(data.asin.unique())
user_unique = sorted(data.reviewerID.unique())
print("# num of user: %d \n# num of item: %d"%(len(user_unique), len(item_unique)))

# num of user: 1686 
# num of item: 962


In [28]:
# Group data following user

n_user = len(user_unique)
n_item = len(item_unique)

previous_user_id = 0
ratings = [0] * n_user
f = open("%s/ratings.txt"%data_dir, "w")
for _, r in data.iterrows():
    uid = user_unique.index(r.reviewerID)
    iid = item_unique.index(r.asin)
    if ratings[uid] == 0:
        ratings[uid] = [[iid, r.overall, r.reviewTime]]
    else:
        ratings[uid].append([iid, r.overall, r.reviewTime])
    f.write("%d::%i::%d::%s\n"%(uid, iid, int(r.overall), r.reviewTime))
f.close()

In [13]:
print("Max item user rated: %d"%max([len(i) for i in ratings]))
print("Min item user rated: %d"%min([len(i) for i in ratings]))
print("Mean item user rated: %d"%np.mean([len(i) for i in ratings]))

Max item user rated: 66
Min item user rated: 5
Mean item user rated: 7


In [16]:
# write new id of items into file
f = open("%s/item_id.txt"%data_dir, "w")
f.write("\n".join(item_unique))
f.close()

In [17]:
# write new id of users into file
f = open("%s/user_id.txt"%data_dir, "w")
f.write("\n".join(user_unique))
f.close()

# Prepare Train, test

In [20]:
shuffle_id = np.random.permutation(n_user)
train_len = int(0.7*n_user)
train_id = shuffle_id[:train_len]
test_id = shuffle_id[train_len:]

ftrain = open("%s/implicit/train.txt"%data_dir, "w")
for idx in train_id:
    user = np.array(ratings[idx]).reshape((len(ratings[idx]), 3))
    user = user[np.argsort(user[:, 2])]
    item = list(user[:, 0])
    item = [str(i) for i in item]
    ftrain.write("%d %s\n"%(idx, " ".join(item)))
ftrain.close()

ftest = open("%s/implicit/test.txt"%data_dir, "w")
for idx in test_id:
    user = np.array(ratings[idx]).reshape((len(ratings[idx]), 3))
    user = user[np.argsort(user[:, 2])]
    item = list(user[:, 0])
    item = [str(i) for i in item]
    ftest.write("%d %s\n"%(idx, " ".join(item)))
ftest.close()

## Create item description for cf-vae

In [33]:
data_item = getDF("%s/meta.json.gz"%data_dir)
data_item.head()

,related,description,categories,title,salesRank,imUrl,asin,price,brand
0,"{'also_viewed': ['0761149430', '0761120149', '...",Steven Raichlen's Best of Barbecue Primal Gril...,"[[Patio, Lawn & Garden, Grills & Outdoor Cooki...","Primal Grill with Steven Raichlen, Volume One",{'Movies & TV': 231134},http://ecx.images-amazon.com/images/I/51cNn5Dl...,0981850006,NaN,NaN
1,"{'also_viewed': ['B008WC0X0A', 'B000CPMOVG', '...",The Tesoro Sand Shark metal combines time-prov...,"[[Patio, Lawn & Garden, Lawn Mowers & Outdoor ...",Sand Shark Margare Maron Audio CD,"{'Patio, Lawn & Garden': 188289}",http://ecx.images-amazon.com/images/I/31B9X0S6...,144072007X,577.15,Tesoro
2,"{'also_viewed': ['1554701511', '1554702720'], ...",This definitive guide to incorporating planks ...,"[[Patio, Lawn & Garden, Grills & Outdoor Cooki...",Napoleon's Everyday Plank Grilling,"{'Patio, Lawn & Garden': 196610}",http://ecx.images-amazon.com/images/I/51cTWw9d...,1554701503,26.39,Napoleon
3,"{'also_viewed': ['1250039479', '1250010160', '...",This is a Bad Kitty backpack pull clip.,"[[Patio, Lawn & Garden, Picnic Baskets & Acces...",Bad Kitty Backpack Pull,{'Toys & Games': 183251},http://ecx.images-amazon.com/images/I/41nR62o%...,1579822932,3.65,Bad Kitty
4,"{'also_viewed': ['B005EN5008', 'B007ZU4BV2', '...",Every spa needs to be shocked to eliminate und...,"[[Patio, Lawn & Garden, Pools, Hot Tubs & Supp...",5 pound SpaGuard Enhanced Spa Shock,"{'Patio, Lawn & Garden': 80211}",http://ecx.images-amazon.com/images/I/21cOwSPt...,1754164498,29.99,SpaGuard


In [35]:
data_item = data_item[data_item.asin.isin(item_unique)]
print(len(data_item))


962


In [80]:
text = []
list_cat = []
for i in item_unique:
    d = data_item[data_item.asin == i]
    text += (d.title + d.description).tolist()
    list_cat.append(d.categories.tolist())

In [72]:
# write file
text = [gensim.utils.simple_preprocess(str(t)) for t in text]
text = [' '.join(t) for t in text]
f = open("%s/description_fix.txt"%data_dir, "w")
f.write("\n".join(text))
f.close()

# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text)
save_npz("%s/item.npz"%data_dir, X)

## Matching with categories

In [84]:
categories = data_item.categories.tolist()
categories = [i for cat in categories for c in cat for i in c]
categories = list(set(categories))
len(categories)

f = open("%s/categories.txt"%data_dir, "w")
for c in list_cat:
    arr = ['0']*len(categories)
    for i in c[0][0]:
        arr[categories.index(i)] = '1'
    f.write(",".join(arr))
f.close()
    

In [88]:
l = [1,2,3,4]
l[-4:]

[1, 2, 3, 4]

In [86]:
os.makedirs("data/test")